## Main Idea
This notebook provides framework for the stock assessment based on the principles of Benjamin Graham. He buys financially strong businesses that are undervalued. Here is the criteria's to which Benjamin selects the stocks to buy:

1. P/E ratio as metric for market temperature. P/E <= 15
2. Company size. Market cap over $10 billion.
3. Good dividend record. Company should be paying dividends at least 10 consecutive years.
4. Price-to-book <= 1.5
5. EPS over 10 years must grow ate least +33%.
6. Current ratio >= 1.5. It's the ratio between current assets and current liabilities.
7. Equity-to-total capital >= 50 %.
8. Debt <= 110% of net current assets
9. Long-term debt <= 50% of total capital
10. Margin of safety at least 30%. It's calculated by the formula

---
V = EPS * (8,5 + 2g)

Where:

* V = intrinsic value (per share)
* EPS = earnings per share (trailing 12 months)
* g = expected annual growth rate (%) over 7–10 years

---
g = (EPSend/EPSstart) ^ (1/n) - 1

Where:
* n = number of years
---
MoS = (V-P)/V

Where:
* P = current market price






#### Data collection and Metrics Calcualtion

**Ticker object creation and market cap**

In [7]:
import yfinance as yf
import pandas as pd

# Apple for example usage
ticker = "AAPL"
stock = yf.Ticker(ticker)

# Collection of calculated metrics
foundational_metrics = {}

# Market cap
foundational_metrics["market_cap"] = stock.fast_info.market_cap


{'market_cap': 3512776855559.0625}


Market cap is higher than \$10 billion dollars, which goes under the criteria

**P/E calculation**

In [40]:
# Latest stock price
price = stock.fast_info.last_price

# Income statement (annual)
income_stmt = stock.income_stmt.T

# Extract Net Income
net_income = income_stmt["Net Income"]

# Balance sheet for shares outstanding
balance_sheet = stock.balance_sheet.T
shares_out = balance_sheet["Ordinary Shares Number"]

# Compute EPS = Net Income / Shares
eps = net_income / shares_out

eps = eps.dropna().sort_index()  # clean
eps_last3 = eps.tail(3)          # last 3 years
eps_avg3 = eps_last3.mean()

foundational_metrics["P/E"] = round(price/eps_avg3, 2)

print("3-year average EPS:", round(eps_avg3, 2))
print(f"Graham P/E:{round(price/eps_avg3, 2)}")

3-year average EPS: 6.23
Graham P/E:36.79


It means that you'll pay \$36  for \$1 of annual earnings which is too much according to above mentioned criteria (<=15)

**Dividends**

In [42]:
# Data about dividens
dividends = stock.dividends

# Calculation dividends by each year
dividends = stock.dividends

# Group by year, sum total dividends per year
div_by_year = dividends.groupby(dividends.index.year).sum()

foundational_metrics["dividends_for_past_10year"] = True

# Dividends for the past 10 years
div_by_year.tail(10)

,Dividends
Date,
2016,0.5575
2017,0.6150
2018,0.7050
2019,0.7600
2020,0.8075
2021,0.8650
2022,0.9100
2023,0.9500
2024,0.9900


For the past ten years Apple didn't miss a year of paying dividends. For the past year apple payed about \$1 for owning 1 stock.

**Price to book**

In [41]:
bs = stock.balance_sheet.T  # transpose → years as rows
equity = bs["Stockholders Equity"].iloc[0]  # latest year

# Shares Outstanding
shares_out = stock.fast_info.shares

# Book Value per Share
bvps = equity / shares_out

# Price-to-Book
pb_ratio = price / bvps

print("Price:", round(price,2))
print("Book Value per Share:", round(bvps, 2))
print("P/B Ratio:", round(pb_ratio, 2))

foundational_metrics["P/B"] = round(pb_ratio, 2)

Price: 229.31
Book Value per Share: 3.72
P/B Ratio: 61.68


P/B ratio is higher than 1. Which means that market values 61 times the stock than it's accounting equity

**Compound EPS for 10 years**
Data collected from:
https://www.macrotrends.net/stocks/charts/AAPL/apple/eps-earnings-per-share-diluted

In [45]:
last_year_eps = 6.08 # for 2024
past_10_year_eps = 1.61 # for 2014

# Calculating for 10 years
eps_growth_10years = ((last_year_eps/past_10_year_eps) - 1) * 100

foundational_metrics["EPS_growth_10years"] = round(eps_growth_10years, 2)
print(f"EPS growth for past 10 years is: {round(eps_growth_10years, 2)}%")

EPS growth for past 10 years is: 277.64%


Earnings per shar rose for 277% for past 10 years, which is passes the >33% condition.

**Current ratio**

In [53]:
current_assets = bs["Current Assets"].iloc[0]
current_liabilities = bs["Current Liabilities"].iloc[0]

# Current ratio
current_ratio = current_assets / current_liabilities
print("Current Assets:", current_assets)
print("Current Liabilities:", current_liabilities)
print("Current Ratio:", round(current_ratio, 2))

foundational_metrics["current_ratio"] = round(current_ratio, 2)

Current Assets: 152987000000.0
Current Liabilities: 176392000000.0
Current Ratio: 0.87


The current ratio is lower than 1, which means it has more liabilities than assets.

**Equity-to-total capital**

In [57]:
# Shareholder equity
equity = bs["Stockholders Equity"].iloc[0]

# Total Debt
debt = bs["Total Debt"].iloc[0]

equity_to_total_capital = equity / (equity + debt)

foundational_metrics["equity_to_total_capital"] = round(equity_to_total_capital, 2)

print(f"ETTC: {round(equity_to_total_capital, 2)}")

ETTC: 0.35


This metrics shows us that 65% of total capital is funded by the debt, which is not safe from the perspective of the value investor.

**Debt and Long-term debt**

In [61]:
# Check for the debt conditions
net_current = current_assets - current_liabilities
debt_test =  debt <= 1.1 * net_current
foundational_metrics["debt<=110%"] = debt_test

# Check for the long term debt
long_term_debt = bs["Long Term Debt"].iloc[0]
total_capital = equity + long_term_debt
long_debt_test = long_term_debt <= 0.5 * total_capital
foundational_metrics["long_term_debt<=50%"] = long_debt_test

**Margin of Safety**


In [68]:
 g = (last_year_eps/past_10_year_eps) ** (1/10) - 1
 v = eps_avg3 * (8.5 + 2 * g * 100)
 mos = (v - price)/ v

 print(f"Expected Annual Growth Rate: {round(g * 100, 2)}%")
 print(f"Intrinsic value of the stock: {round(v, 2)} $")
 print(f"Margin of Safety {mos}")


 foundational_metrics["intrinsic_value"] = round(v, 2)
 foundational_metrics["margin_of_safety"] = round(g * 100, 2)

Expected Annual Growth Rate: 14.21%
Intrinsic value of the stock: 230.12 $
Margin of Safety 0.003539522223471661


Intrinsic value of the stock is \$230, which is only \$1 difference between real price. This means it's fairly priced.

#### Summary
Apple stock according to the principles of Graham is not suitable for buying. Currently it's fairly priced and only goes 3 conditions out of 10, which is not good.


**The idea of this feature it will fundamentally assess the stock according to the fundamentals and tell if it's overpriced, under praiced or fairly prices with intrinsic value of the stock.**

In [74]:
fm = pd.DataFrame([foundational_metrics])

In [73]:
fm

,market_cap,P/E,P/B,dividends_for_past_10year,EPS_growth_10years,current_ratio,equity_to_total_capital,debt<=110%,long_term_debt<=50%,intrinsic_value,margin_of_safety
0,3.512777e+12,36.79,61.68,True,277.64,0.87,0.35,False,False,230.12,14.21
